# 0. Environment

## 0-1. check and install

In [ ]:
!python --version

In [ ]:
!sudo apt update
!sudo apt install wget -y
!sudo apt install libsndfile1-dev -y

In [ ]:
!python -m pip install --upgrade pip 
!python -m pip install flatbuffers==24.3.6
!python -m pip install tensorflow==2.15.0
!python -m pip install librosa
!python -m pip install python_speech_features
!python -m pip install seaborn

In [ ]:
import tensorflow as tf
print('tensorflow version:', tf.__version__)

## 0-2. set parameter

In [ ]:
# set dataset path
dataset_path = './data_speech_commands_v002'

# 1. Prepare dataset

## 1-1. get dataset

In [ ]:
import os
# get speech_commands from google's server
if not os.path.exists(dataset_path):
    if not os.path.isfile('speech_commands_v0.02.tar.gz') :
        !wget https://storage.googleapis.com/download.tensorflow.org/data/speech_commands_v0.02.tar.gz
    !mkdir data_speech_commands_v002
    !tar zxf speech_commands_v0.02.tar.gz --checkpoint=1000 --checkpoint-action="echo=%T Unpacking... (%u/333756)" --directory data_speech_commands_v002

## 1-2. dataset details

In [ ]:
import numpy as np
# list the folders in the dataset
dataset_dirslist = np.array(tf.io.gfile.listdir(str(dataset_path)))
print ('dataset_dirslist:', dataset_dirslist,', num: ' , len(dataset_dirslist))
print('-')

# delete the files and folders which don't contain the trainable command folders from the list
dataset_commands_dirslist = dataset_dirslist
dataset_commands_dirslist = dataset_commands_dirslist[dataset_commands_dirslist != 'README.md']
dataset_commands_dirslist = dataset_commands_dirslist[dataset_commands_dirslist != '.DS_Store']
dataset_commands_dirslist = dataset_commands_dirslist[dataset_commands_dirslist != 'validation_list.txt']
dataset_commands_dirslist = dataset_commands_dirslist[dataset_commands_dirslist != 'testing_list.txt']
dataset_commands_dirslist = dataset_commands_dirslist[dataset_commands_dirslist != 'LICENSE']
dataset_commands_dirslist = dataset_commands_dirslist[dataset_commands_dirslist != '_background_noise_']
print ('dataset_commands_dirslist:', dataset_commands_dirslist,', num: ' , len(dataset_commands_dirslist))

## 1-3. dataset commands filepaths

In [ ]:
# list the num of audio files in each command folder
dataset_commands_filepaths = []
for item in dataset_commands_dirslist:
    dataset_commands_dir = os.path.join(dataset_path, item,'*')
    print('dataset_commands_dir:', dataset_commands_dir)
    fileslist = tf.io.gfile.glob(dataset_commands_dir)
    print('file num of',item,':', len(fileslist))
    dataset_commands_filepaths.extend(fileslist)
    print('-')
print('num of dataset_commands_filepaths:', len(dataset_commands_dirslist))
print('num of dataset_commands_filepaths:', len(dataset_commands_filepaths))
print('sample of dataset_commands_filepaths:', dataset_commands_filepaths[0])

# 2. Audio Preporcess

## 2-1. function

### 2-1-1. split num

In [ ]:
# get the start index to split the dataset  
def get_split_num(train_ratio = 0.8, val_ratio = 0.1, test_ratio = 0.1, file_num = 0):
    #get train , val, test num
    train_num = int( file_num * train_ratio )
    val_num = int( file_num * val_ratio )
    test_num = file_num - train_num - val_num 

    #get train , val, test first_num
    train_first_num = 0
    val_first_num = train_num
    test_first_num = train_num + val_num
    
    return train_first_num, val_first_num, test_first_num

### 2-1-2. label

In [ ]:
# get label from the filepath of audio file
def get_label(filepath):
    split_1=os.path.split(filepath)
    split_2=os.path.split(split_1[0])
    return split_2[1]

In [ ]:
tmp = get_label(dataset_commands_filepaths[0])
print ('get_label(dataset_commands_filepaths[0]):')
print (type(tmp))
print (tmp)

### 2-1-3. waveform

In [ ]:
import librosa
# load waveform from the filepath
def get_waveform(filepath):
    sample_rate = 16000
    waveform, fs = librosa.load(filepath, sr=sample_rate)
    return waveform

In [ ]:
tmp = get_waveform(dataset_commands_filepaths[0])
print ('get_waveform(dataset_commands_filepaths[0]):')
print (type(tmp))
print (tmp.shape)
print (tmp)

### 2-1-4. label id

In [ ]:
# get label id from label and command_list
def get_label_id(label,command_list):
    label_id = np.argmax(label==command_list)
    return label_id

In [ ]:
tmp = get_label_id(get_label(dataset_commands_filepaths[0]),dataset_commands_dirslist)
print ('get_label_id(get_label(dataset_commands_filepaths[0]):')
print (type(tmp))
print (tmp)

### 2-1-5. MFCC-python_speech_features

In [ ]:
import python_speech_features
import matplotlib.pyplot as plt
def get_features(waveform):
    # Zero-padding for an audio waveform with less than 16,000 samples.
    # input_len = 16000
    # waveform = waveform[:input_len]
    # np.shape(waveform) =  (16000,)
    zero_padding = np.zeros( (16000-np.shape(waveform)[0],), dtype=np.float32)
    # Cast the waveform tensors with np.float32
    waveform = np.cast['float32'](waveform)
    # Concatenate the waveform with `zero_padding`, which ensures all audio clips are of the same length.
    equal_length = np.concatenate([waveform, zero_padding], 0)
    
    sample_rate = 16000
    num_mfcc = 16
    len_mfcc = 40
    mfccs = python_speech_features.base.mfcc(equal_length, 
                                            samplerate=sample_rate,
                                            winlen=0.256,
                                            winstep=0.050,
                                            numcep=num_mfcc,
                                            nfilt=26,
                                            nfft=4096,
                                            preemph=0.0,
                                            ceplifter=0,
                                            appendEnergy=False,
                                            winfunc=np.hanning)
    return mfccs.transpose()

In [ ]:
tmp = get_features(get_waveform(dataset_commands_filepaths[0]))
print ('get_features(get_waveform(dataset_commands_files[0])):')
print ('type:',type(tmp))
print ('shape:',tmp.shape)
#print (tmp)
fig = plt.figure()
plt.imshow(tmp, cmap='inferno', origin='lower')
plt.show()

## 2-2. Preporcess set commands

In [ ]:
x_train = []
y_train = []
x_val = []
y_val = []
x_test = []
y_test = []

### 2-2-1. get set command filepaths

In [ ]:
import random
random.seed(66)

# set want commands
commands = np.array(['on','stop'])

train_set_commands_filepaths = []
val_set_commands_filepaths = []
test_set_commands_filepaths = []

# get set commands
set_commands_dirslist = commands
# get the filepaths and split to train, val, test every command
for item in set_commands_dirslist:
    # join the dataset_path and command
    set_commands_dir = os.path.join(dataset_path, item,'*')
    print('set_commands_dir:', set_commands_dir)
    
    # list the set command filepaths in each dir
    fileslist = tf.io.gfile.glob(set_commands_dir)
    random.shuffle(fileslist)
    print('file num :', len(fileslist))
    print('-')
    
    # split the filslist to train, val, test
    train, val, test = get_split_num(0.8, 0.1, 0.1, len(fileslist))
    train_set_commands_filepaths.extend(fileslist[:val])
    val_set_commands_filepaths.extend(fileslist[val:test])
    test_set_commands_filepaths.extend(fileslist[test:])  
# get the num of all tarinable audio files
total_set_commands_num = len(train_set_commands_filepaths)+len(val_set_commands_filepaths)+len(test_set_commands_filepaths)
    
print('num of train_set_commands_filepaths:', len(train_set_commands_filepaths))
print('num of val_set_commands_filepaths:', len(val_set_commands_filepaths))
print('num of test_set_commands_filepaths:', len(test_set_commands_filepaths))

print('total of set_commands_filepaths:',total_set_commands_num)
print('sample of train_set_commands_filepaths:', train_set_commands_filepaths[0])


### 2-2-2. get features and label id

In [ ]:
train_set_commands_features = []
val_set_commands_features = []
test_set_commands_features = []

train_set_commands_label_id = []
val_set_commands_label_id = []
test_set_commands_label_id = []
 
# get features and label id and append to train_set_commands_features and train_set_commands_label_id
for i, item in enumerate(train_set_commands_filepaths):
    waveform = get_waveform(item)
    features = get_features(waveform)
    train_set_commands_features.append(features)

    label = get_label(item)
    label_id = get_label_id(label,commands)
    train_set_commands_label_id.append(label_id)

    if (i+1)%10 == 0:
        print(f'\rget features and label id and append to train set commands... ({i+1}/{len(train_set_commands_filepaths)})', end='')
if (i+1)%10 != 0:
    print(f'\rget features and label id and append to train set commands... ({i+1}/{len(train_set_commands_filepaths)})')

# get features and label id and append to val_set_commands_features and val_set_commands_label_id
for i, item in enumerate(val_set_commands_filepaths):
    waveform = get_waveform(item)
    features = get_features(waveform)
    val_set_commands_features.append(features)

    label = get_label(item)
    label_id = get_label_id(label,commands)
    val_set_commands_label_id.append(label_id)

    if (i+1)%10 == 0:
        print(f'\rget features and label id and append to validation set commands... ({i+1}/{len(val_set_commands_filepaths)})', end='')
if (i+1)%10 != 0:
    print(f'\rget features and label id and append to validation set commands... ({i+1}/{len(val_set_commands_filepaths)})')

# get features and label id and append to test_set_commands_features and test_set_commands_label_id    
for i, item in enumerate(test_set_commands_filepaths):
    waveform = get_waveform(item)
    features = get_features(waveform)
    test_set_commands_features.append(features)

    label = get_label(item)
    label_id = get_label_id(label,commands)
    test_set_commands_label_id.append(label_id)        
        
    if (i+1)%10 == 0:
        print(f'\rget features and label id and append to test set commands...  ({i+1}/{len(test_set_commands_filepaths)})', end='')
if (i+1)%10 != 0:
    print(f'\rget features and label id and append to test set commands...  ({i+1}/{len(test_set_commands_filepaths)})')

print('-')
print ('num of train_set_commands_features:', len(train_set_commands_features))
print ('num of train_set_commands_label_id:', len(train_set_commands_label_id))
print('-')
print ('num of val_set_commands_features:', len(val_set_commands_features))
print ('num of val_set_commands_label_id:', len(val_set_commands_label_id))
print('-')
print ('num of test_set_commands_features:', len(test_set_commands_features))
print ('num of test_set_commands_label_id:', len(test_set_commands_label_id)) 

### 2-2-3. add features and label id to dataset

In [ ]:
# add features and label id to train list
x_train.extend(train_set_commands_features)
y_train.extend(train_set_commands_label_id)

# add features and label id to val list
x_val.extend(val_set_commands_features)
y_val.extend(val_set_commands_label_id)

# add features and label id to test list
x_test.extend(test_set_commands_features)
y_test.extend(test_set_commands_label_id)

print('num of x_train:',len(y_train))
print('num of y_train:',len(y_train))
print('-')
print('num of x_val:',len(x_val))
print('num of y_val:',len(y_val))
print('-')
print('num of x_test:',len(x_test))
print('num of y_test:',len(y_test))

## 2-3. Preporcess unknown commands

### 2-3-1. get unknown command filepaths

In [ ]:
train_unknown_commands_filepaths = []
val_unknown_commands_filepaths = []
test_unknown_commands_filepaths = []

# add unknown to commands list
if not np.any(commands=='unknown'):
    commands = np.append(commands,'unknown')
print ('commands:',commands)
print('-')

# get the unknown commands which are not set commands and add them to the list
unknown_commands_dirslist = np.setdiff1d( dataset_commands_dirslist , commands)
print ('unknown_commands_dirslist:',unknown_commands_dirslist)
print('-')

# set the ratio of unknown command filepaths to set command filepaths
unknown_ratio = 0.1
print('unknown_ratio:',unknown_ratio)
print ('total_set_commands_num:', total_set_commands_num)
# get the num of unknown command filepaths
unknown_command_num = int(total_set_commands_num * unknown_ratio)
print ('unknown_command_num:', unknown_command_num)
# get the the num of each unknown command filepaths
each_unknown_command_num = unknown_command_num//len(unknown_commands_dirslist)
print ('each_unknown_command_num:', each_unknown_command_num)
print('-')

# get the filepaths and split to train, val, test every command
for item in unknown_commands_dirslist:
    # join the dataset_path and command
    unknown_commands_dir = os.path.join(dataset_path, item,'*')
    print('unknown_commands_dir:', unknown_commands_dir)
    
    # list the unknown command filepaths in each dir
    fileslist = tf.io.gfile.glob(unknown_commands_dir)
    random.shuffle(fileslist)
    print('file num :', len(fileslist))
    
    # split the filslist to train, val, test
    train, val, test = get_split_num(0.8, 0.1, 0.1, each_unknown_command_num)
    train_unknown_commands_filepaths.extend(fileslist[:val])
    val_unknown_commands_filepaths.extend(fileslist[val:test])
    test_unknown_commands_filepaths.extend(fileslist[test:each_unknown_command_num])
    print('-')
    
print('num of train_unknown_commands_filepaths:', len(train_unknown_commands_filepaths))
print('num of val_unknown_commands_filepaths:', len(val_unknown_commands_filepaths))
print('num of test_unknown_commands_filepaths:', len(test_unknown_commands_filepaths))
print('sample of train_unknown_commands_filepaths:', train_unknown_commands_filepaths[0])

### 2-3-2. get features and label id

In [ ]:
train_unknown_commands_features = []
val_unknown_commands_features = []
test_unknown_commands_features = []

train_unknown_commands_label_id = []
val_unknown_commands_label_id = []
test_unknown_commands_label_id = []

# get the unknown label and label_id
label = 'unknown'
label_id = get_label_id(label,commands)

# get features and label id and append to train_unknown_commands_features and train_unknown_commands_label_id
for i, item in enumerate(train_unknown_commands_filepaths):
    waveform = get_waveform(item)
    features = get_features(waveform)
    train_unknown_commands_features.append(features)
    train_unknown_commands_label_id.append(label_id)

    if (i+1)%10 == 0:
        print(f'\rget features and label id and append to train unknown commands...  ({i+1}/{len(train_unknown_commands_filepaths)})', end='')
if (i+1)%10 != 0:
    print(f'\rget features and label id and append to train unknown commands...  ({i+1}/{len(train_unknown_commands_filepaths)})')

# get features and label id and append to val_unknown_commands_features and val_unknown_commands_label_id
for i, item in enumerate(val_unknown_commands_filepaths):
    waveform = get_waveform(item)
    features = get_features(waveform)
    val_unknown_commands_features.append(features)
    val_unknown_commands_label_id.append(label_id)

    if (i+1)%10 == 0:
        print(f'\rget features and label id and append to validation unknown commands...  ({i+1}/{len(val_unknown_commands_filepaths)})', end='')
if (i+1)%10 != 0:
    print(f'\rget features and label id and append to validation unknown commands...  ({i+1}/{len(val_unknown_commands_filepaths)})')

# get features and label id and append to test_unknown_commands_features and test_unknown_commands_label_id
for i, item in enumerate(test_unknown_commands_filepaths):
    waveform = get_waveform(item)
    features = get_features(waveform)
    test_unknown_commands_features.append(features)
    test_unknown_commands_label_id.append(label_id)        

    if (i+1)%10 == 0:
        print(f'\rget features and label id and append to test unknown commands...  ({i+1}/{len(test_unknown_commands_filepaths)})', end='')
if (i+1)%10 != 0:
    print(f'\rget features and label id and append to test unknown commands...  ({i+1}/{len(test_unknown_commands_filepaths)})')    
print('-')
print ('num of train_unknown_commands_features:', len(train_unknown_commands_features))
print ('num of train_unknown_commands_label_id:', len(train_unknown_commands_label_id))
print('-')
print ('num of val_unknown_commands_features:', len(val_unknown_commands_features))
print ('num of val_unknown_commands_label_id:', len(val_unknown_commands_label_id))
print('-')
print ('num of test_unknown_commands_features:', len(test_unknown_commands_features))
print ('num of test_unknown_commands_label_id:', len(test_unknown_commands_label_id)) 

### 2-3-3. add features and label id to dataset

In [ ]:
# add features and label id to train list
x_train.extend(train_unknown_commands_features)
y_train.extend(train_unknown_commands_label_id)
# add features and label id to val list
x_val.extend(val_unknown_commands_features)
y_val.extend(val_unknown_commands_label_id)
# add features and label id to test list
x_test.extend(test_unknown_commands_features)
y_test.extend(test_unknown_commands_label_id)

print('num of x_train:',len(x_train))
print('num of y_train:',len(y_train))
print('-')
print('num of x_val:',len(x_val))
print('num of y_val:',len(y_val))
print('-')
print('num of x_test:',len(x_test))
print('num of y_test:',len(y_test))

## 2-4. Preporcess slience command

### 2-4-1. get background command filepaths

In [ ]:
train_slience_commands_filepaths = []
val_slience_commands_filepaths = []
test_slience_commands_filepaths = []

# add slience command to commands list
if not np.any(commands=='slience'):
    commands = np.append(commands,'slience')
print ('commands:',commands)
print('-')

# get background_noise to contruct slience samples
slience_commands_dirslist = ['_background_noise_']
print ('slience_commands_dirslist:',slience_commands_dirslist)
print('-')

# set the ratio of slience samples to set command filepaths
slience_ratio = 0.1
print ('total_set_commands_num:', total_set_commands_num)
print('slience_ratio:',slience_ratio)
# get the num of slience samples
slience_command_num = int(total_set_commands_num * slience_ratio)
print ('slience_command_num:', slience_command_num)
print('-')

# get the background_noise filepaths and append to train, val, test every filepaths
for item in slience_commands_dirslist:
    # join the dataset_path and background_noise files
    slience_commands_dir = os.path.join(dataset_path, item,'*.wav')
    print('slience_commands_dir:', slience_commands_dir)
    
    # get list of background_noise files
    fileslist = tf.io.gfile.glob(slience_commands_dir)
    print('file num :', len(fileslist))
    
    # get num of each background_noise files (the num to append for each files)
    each_slience_files = slience_command_num//len(fileslist)
    print ('each_slience_files:', each_slience_files)
  
    # get num of tarin , val ,test for each files
    train_num = int(each_slience_files*0.8)
    val_num = int(each_slience_files*0.1)
    test_num = int(each_slience_files*0.1)
    print('-')
    
    # every background_noise file
    for itme in fileslist:
        # append the filepath to train,val,test list for each files
        for i in range(train_num):
            train_slience_commands_filepaths.append(itme)
        for i in range(val_num):
            val_slience_commands_filepaths.append(itme)
        for i in range(test_num):
            test_slience_commands_filepaths.append(itme)

print('num of train_slience_commands_filepaths:', len(train_slience_commands_filepaths))
print('num of val_slience_commands_filepaths:', len(val_slience_commands_filepaths))
print('num of test_slience_commands_filepaths:', len(test_slience_commands_filepaths))
print('sample of train_slience_commands_filepaths:', train_slience_commands_filepaths[0])

### 2-4-2. contruct slience features and label id

In [ ]:
#  ontruct slience features from background_noise
def get_random_background_noise_waveform(filepath):
    # get waveform and len from filepath
    waveform = get_waveform(filepath)  
    waveform_len = waveform.shape[0]
    # get start time by random(waveform_len)
    split_start_time = random.randrange(0, waveform_len)
    # get the 1s background_noise sample to represent slience feature
    waveform_split = waveform[split_start_time:split_start_time+16000]
    return waveform_split

In [ ]:
# contruct slience features by zero vector
def get_zero_waveform():
    # get the 1s zero vector sample to represent slience feature
    zero_waveform = np.zeros((16000,), dtype=np.float32)
    return zero_waveform

In [ ]:
train_slience_commands_features = []
val_slience_commands_features = []
test_slience_commands_features = []

train_slience_commands_label_id = []
val_slience_commands_label_id = []
test_slience_commands_label_id = []

# get the slience label and label_id
label = 'slience'
label_id = get_label_id(label,commands)

# get features and label id by get_random_background_noise_waveform and append them to train list
for i, item in enumerate(train_slience_commands_filepaths):
    background_noise_waveform = get_random_background_noise_waveform(item)
    features = get_features(background_noise_waveform)
    train_slience_commands_features.append(features)
    train_slience_commands_label_id.append(label_id)
    if (i+1)%10 == 0:
        print(f'\rget features and label id and append to train slience commands...  ({i+1}/{len(train_slience_commands_filepaths)})', end='')
if (i+1)%10 != 0:
    print(f'\rget features and label id and append to train slience commands...  ({i+1}/{len(train_slience_commands_filepaths)})')    
# get features and label id by get_zero_waveform and append one sample to train list
zero_waveform = get_zero_waveform()
features = get_features(zero_waveform)
train_slience_commands_features.append(features)
train_slience_commands_label_id.append(label_id)

# get features and label id by get_random_background_noise_waveform and append them to val list
for i, item in enumerate(val_slience_commands_filepaths):
    background_noise_waveform = get_random_background_noise_waveform(item)
    features = get_features(background_noise_waveform)
    val_slience_commands_features.append(features)
    val_slience_commands_label_id.append(label_id)
    if (i+1)%10 == 0:
        print(f'\rget features and label id and append to validation slience commands...  ({i+1}/{len(val_slience_commands_filepaths)})', end='')
if (i+1)%10 != 0:
    print(f'\rget features and label id and append to validation slience commands...  ({i+1}/{len(val_slience_commands_filepaths)})')      
# get features and label id by get_zero_waveform and append one sample to val list
zero_waveform = get_zero_waveform()
features = get_features(zero_waveform)
val_slience_commands_features.append(features)
val_slience_commands_label_id.append(label_id)

# get features and label id by get_random_background_noise_waveform and append them to test list
for i, item in enumerate(test_slience_commands_filepaths):
    background_noise_waveform = get_random_background_noise_waveform(item)
    features = get_features(background_noise_waveform)
    test_slience_commands_features.append(features)
    test_slience_commands_label_id.append(label_id) 
    if (i+1)%10 == 0:
        print(f'\rget features and label id and append to test slience commands...  ({i+1}/{len(test_slience_commands_filepaths)})', end='')
if (i+1)%10 != 0:
    print(f'\rget features and label id and append to test slience commands...  ({i+1}/{len(test_slience_commands_filepaths)})')     
# get features and label id by get_zero_waveform and append one sample to test list
zero_waveform = get_zero_waveform()
features = get_features(zero_waveform)
test_slience_commands_features.append(features)
test_slience_commands_label_id.append(label_id)
print('-')
print ('num of train_slience_commands_features:', len(train_slience_commands_features))
print ('num of train_slience_commands_label_id:', len(train_slience_commands_label_id))
print('-')
print ('num of val_slience_commands_features:', len(val_slience_commands_features))
print ('num of val_slience_commands_label_id:', len(val_slience_commands_label_id))
print('-')
print ('num of test_slience_commands_features:', len(test_slience_commands_features))
print ('num of test_slience_commands_label_id:', len(test_slience_commands_label_id)) 

### 2-4-3. add features and label id to dataset

In [ ]:
# add features and label id to train list
x_train.extend(train_slience_commands_features)
y_train.extend(train_slience_commands_label_id)
# add features and label id to val list
x_val.extend(val_slience_commands_features)
y_val.extend(val_slience_commands_label_id)
# add features and label id to test list
x_test.extend(test_slience_commands_features)
y_test.extend(test_slience_commands_label_id)

print('num of x_train:',len(x_train))
print('num of y_train:',len(y_train))
print('-')
print('num of x_val:',len(x_val))
print('num of y_val:',len(y_val))
print('-')
print('num of x_test:',len(x_test))
print('num of y_test:',len(y_test))

## 2-5.  save and load processed dataset

In [ ]:
feature_sets_file = 'all_targets_mfcc_sets.npz'
# Save features and truth vector (y) sets to disk
np.savez(feature_sets_file, 
         x_train=x_train, 
         y_train=y_train, 
         x_val=x_val, 
         y_val=y_val, 
         x_test=x_test, 
         y_test=y_test)

In [ ]:
feature_sets_path = './'
feature_sets_filename = 'all_targets_mfcc_sets.npz'
# Load feature sets
feature_sets = np.load(os.path.join(feature_sets_path, feature_sets_filename))
print(feature_sets.files)
# Assign feature sets
x_train = feature_sets['x_train']
y_train = feature_sets['y_train']
x_val = feature_sets['x_val']
y_val = feature_sets['y_val']
x_test = feature_sets['x_test']
y_test = feature_sets['y_test']

# 3. Train

## 3-1. prepare input

In [ ]:
# zip and shuffle 
train_list = list(zip(x_train, y_train))
random.shuffle(train_list)
x_train, y_train = zip(*train_list)

val_list = list(zip(x_val, y_val))
random.shuffle(val_list)
x_val, y_val = zip(*val_list)

test_list = list(zip(x_test, y_test))
random.shuffle(test_list)
x_test, y_test = zip(*test_list)

# trans to np.array to expand dims
x_tn = np.array(x_train)
y_tn = np.array(y_train)
x_v = np.array(x_val)
y_v = np.array(y_val)
x_ts = np.array(x_test)
y_ts = np.array(y_test)

# expand dims to satifiy input shape
x_tn = np.expand_dims(x_tn, axis=-1)
x_v = np.expand_dims(x_v, axis=-1)
x_ts = np.expand_dims(x_ts, axis=-1)

print (x_tn.shape)
print (y_tn.shape)
print (x_v.shape)
print (y_v.shape)
print (x_ts.shape)
print (y_ts.shape)



## 3-2. set parameter

In [ ]:
# get and check input shape by sampling 
tmp = get_features(get_waveform(dataset_commands_filepaths[0]))
tmp = np.expand_dims(tmp, axis=-1)
input_shape = tmp.shape
print('Input shape:', input_shape)
# get and check output shape by num of commands
num_labels = len(commands)
print('Output shape:', num_labels)

## 3-3. build model

In [ ]:
import tensorflow.keras
from tensorflow.keras import layers
from tensorflow.keras import models

# build a model
model = models.Sequential([
    layers.Input(shape=input_shape),
    layers.Conv2D(32, 3, activation='relu'),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.1),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.1),
    layers.Dense(num_labels),
])

model.summary()

In [ ]:
# compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

## 3-4. train

In [ ]:
# train the model
EPOCHS = 20
history = model.fit(
    x_tn,y_tn,
    validation_data=(x_v,y_v),
    epochs=EPOCHS,
    batch_size=64,
    callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=3),
)

## 3-5. result

In [ ]:
import matplotlib.pyplot as plt
# determine overfitting by checking the metrics
metrics = history.history
plt.plot(history.epoch, metrics['loss'], metrics['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.show()

## 3-6. save trained model

In [ ]:
# set model file name
model_filename = 'SpeechCommandRecognition_model.h5'
# Save the model as a file
models.save_model(model, model_filename)

## 3-7. evaluate

In [ ]:
# inference on test set
y_pred = np.argmax(model.predict(x_ts), axis=1)
#print(y_pred)
y_true = y_ts
#print (y_true)
test_acc = sum(y_pred == y_true) / len(y_true)
print(f'Test set accuracy: {test_acc:.0%}')

## 3-8. display a confusion matrix

In [ ]:
import seaborn as sns
# check the result the confusion_matrix 
confusion_mtx = tf.math.confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_mtx,
            xticklabels=commands,
            yticklabels=commands,
            annot=True, fmt='g')
plt.xlabel('Prediction')
plt.ylabel('Label')
plt.show()

## 3-9. run inference on an audio file

In [ ]:
# run inference on an audio file
audio_file = os.path.join(dataset_path, 'on','0ba018fc_nohash_1.wav')
sample_feature = get_features(get_waveform(audio_file))
sample_feature = np.expand_dims(sample_feature, axis=-1)
sample_feature = np.expand_dims(sample_feature, axis=0)

sample_label = get_label(audio_file)
sample_label_id = get_label_id(sample_label, commands)

prediction = model(sample_feature)
plt.bar(commands, tf.nn.softmax(prediction[0]))
plt.title(f'Predictions for "{commands[sample_label_id]}"')
plt.show()

# 4. Convert

## 4-1. set parameter

In [ ]:
from tensorflow import lite
from tensorflow.keras import models
# set the file name
keras_model_filename = 'SpeechCommandRecognition_model.h5'
tflite_filename = 'SpeechCommandRecognition_model.tflite'
tflite_quantization_filename = 'SpeechCommandRecognition_quantization_model.tflite'

## 4-2. convert (float)

In [ ]:
# Convert the model to the TensorFlow Lite format without quantization
model = models.load_model(keras_model_filename)
converter = lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open(tflite_filename, 'wb').write(tflite_model)

# Download trained model

In [ ]:
from google.colab import files
files.download(tflite_filename)